<a href="https://colab.research.google.com/github/PasAlex8019/Hello-World/blob/master/9_3_%D0%A0%D0%B0%D0%B7%D0%BD%D1%8B%D0%B5_%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D0%B8_%D0%B0%D1%82%D1%80%D0%B8%D0%B1%D1%83%D1%86%D0%B8%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Цель задания

### Научиться рассчитывать ROI для разных моделей атрибуции. 


**Что нужно сделать**

Изучите PDF-документ с дополнительной информацией о рассчёте ROI для разных моделей атрибуции. Проведите расчёты и ответьте на ряд вопросов:

Как зависит ценность сессии от количества сессий у пользователя в линейной модели атрибуции? (чем больше сессий, тем меньше ценность одного визита).

Какая сессия для нас ценнее, согласно модели time-decay: та, которая произошла сутки назад или та, что произошла две недели назад? (ценнее более свежая сессия, которая произошла сутки назад)

В каком случае ценность на одну сессию снижается сильнее, когда у пользователя 5 сессий или когда у него 10 сессий, для модели time-decay? (когда у пользователя 5 сессий, потому что количество сессий находится в знаменателе дроби или потому что во втором случае мы делим ценность на бОльшее количество частей и перепады в ценностях будут ниже).


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

In [9]:
session_data = pd.read_csv('data.csv', sep=';')
session_data.head()

,userId,date,trafficSource,cost,value
0,user_10,2020-01-05,telegram / posts,15.75,215.0
1,user_10000,2020-01-17,yandex / cpc,8.50,0.0
2,user_10000,2020-01-19,google / cpc,8.25,0.0
3,user_1002,2020-01-03,telegram / posts,15.75,0.0
4,user_1003,2020-01-08,google / cpc,8.25,0.0


In [11]:
session_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   userId         10000 non-null  object 
 1   date           10000 non-null  object 
 2   trafficSource  10000 non-null  object 
 3   cost           10000 non-null  float64
 4   value          10000 non-null  float64
dtypes: float64(2), object(3)
memory usage: 390.8+ KB


In [12]:
# колонку 'date' приводим к формату даты
session_data['date'] = [datetime.strptime(x, '%Y-%m-%d') for x in session_data['date']]
session_data

,userId,date,trafficSource,cost,value
0,user_10,2020-01-05,telegram / posts,15.75,215.0
1,user_10000,2020-01-17,yandex / cpc,8.50,0.0
2,user_10000,2020-01-19,google / cpc,8.25,0.0
3,user_1002,2020-01-03,telegram / posts,15.75,0.0
4,user_1003,2020-01-08,google / cpc,8.25,0.0
...,...,...,...,...,...
9995,user_999,2020-01-03,facebook / video,10.25,0.0
9996,user_9992,2020-01-04,google / cpc,8.25,0.0
9997,user_9995,2020-01-05,telegram / posts,15.75,0.0
9998,user_9995,2020-01-21,yandex / cpc,8.50,0.0


In [13]:
session_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   userId         10000 non-null  object        
 1   date           10000 non-null  datetime64[ns]
 2   trafficSource  10000 non-null  object        
 3   cost           10000 non-null  float64       
 4   value          10000 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 390.8+ KB


In [14]:
# группировка и суммирования, чтобы объединить покупки с одинакового канала в один день
session_data = session_data.groupby(['userId', 'date', 'trafficSource'])['cost', 'value'].sum().reset_index()
session_data = session_data.sort_values(by=['userId', 'date'])
session_data.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,userId,date,trafficSource,cost,value
0,user_10,2020-01-05,telegram / posts,15.75,215.0
1,user_10000,2020-01-17,yandex / cpc,8.50,0.0
2,user_10000,2020-01-19,google / cpc,8.25,0.0
3,user_1002,2020-01-03,telegram / posts,15.75,0.0
4,user_1003,2020-01-08,google / cpc,8.25,0.0


Так как time-decay модель учитывает время от даты касания, до даты транзакции, для удобства расчетов нам понадобится столбец с датой покупки, сделаем для этого вспомогательную таблицу только с пользователями и датами их покупок. При этом переименуем столбец date на столбец purchaseDate:

In [15]:
purchases_only = session_data[session_data['value'] > 0][['userId', 'date']]
purchases_only

,userId,date
0,user_10,2020-01-05
10,user_1008,2020-01-09
12,user_1011,2020-01-09
18,user_1016,2020-01-03
19,user_1016,2020-01-15
...,...,...
9881,user_9975,2020-01-08
9887,user_9979,2020-01-08
9891,user_9982,2020-01-30
9894,user_9985,2020-01-10


In [17]:
# через max() убираем дубликаты и оставляем максимальные значения
purchases_only = purchases_only.groupby('userId')['date'].max().reset_index()
purchases_only

,userId,date
0,user_10,2020-01-05
1,user_1008,2020-01-09
2,user_1011,2020-01-09
3,user_1016,2020-01-15
4,user_1022,2020-01-03
...,...,...
821,user_9975,2020-01-08
822,user_9979,2020-01-08
823,user_9982,2020-01-30
824,user_9985,2020-01-10


In [18]:
# переименовываем колонки
purchases_only.columns = ['userId', 'purchaseDate']
purchases_only

,userId,purchaseDate
0,user_10,2020-01-05
1,user_1008,2020-01-09
2,user_1011,2020-01-09
3,user_1016,2020-01-15
4,user_1022,2020-01-03
...,...,...
821,user_9975,2020-01-08
822,user_9979,2020-01-08
823,user_9982,2020-01-30
824,user_9985,2020-01-10


Чтобы посчитать атрибуцию по линейной модели, нам нужно суммарную выручку от пользователя разделить по всем визитам в равных долях, для этого посчитаем общее количество визитов (totalSessions) и общую выручку с пользователя (totalValue). Для атрибуции по первому касанию, нам нужен столбец с порядковым номером визита. Эти расчеты, а также объединение исходных данных с датами покупок из предыдущего пункта представлены ниже:

In [19]:
# группируем по пользователям и считаем количество посещений по дням, добавляем колонку
session_data['totalSessions'] = session_data.groupby('userId')['date'].transform(lambda x: x.count())
session_data

,userId,date,trafficSource,cost,value,totalSessions
0,user_10,2020-01-05,telegram / posts,15.75,215.0,1
1,user_10000,2020-01-17,yandex / cpc,8.50,0.0,2
2,user_10000,2020-01-19,google / cpc,8.25,0.0,2
3,user_1002,2020-01-03,telegram / posts,15.75,0.0,1
4,user_1003,2020-01-08,google / cpc,8.25,0.0,2
...,...,...,...,...,...,...
9899,user_999,2020-01-03,facebook / video,10.25,0.0,1
9900,user_9992,2020-01-04,google / cpc,8.25,0.0,1
9901,user_9995,2020-01-05,telegram / posts,15.75,0.0,2
9902,user_9995,2020-01-21,yandex / cpc,8.50,0.0,2


In [20]:
# добавляем колонку с суммой покупок каждого пользователя
session_data['totalValue'] = session_data.groupby('userId')['value'].transform(lambda x: x.sum())
session_data

,userId,date,trafficSource,cost,value,totalSessions,totalValue
0,user_10,2020-01-05,telegram / posts,15.75,215.0,1,215.0
1,user_10000,2020-01-17,yandex / cpc,8.50,0.0,2,0.0
2,user_10000,2020-01-19,google / cpc,8.25,0.0,2,0.0
3,user_1002,2020-01-03,telegram / posts,15.75,0.0,1,0.0
4,user_1003,2020-01-08,google / cpc,8.25,0.0,2,0.0
...,...,...,...,...,...,...,...
9899,user_999,2020-01-03,facebook / video,10.25,0.0,1,0.0
9900,user_9992,2020-01-04,google / cpc,8.25,0.0,1,0.0
9901,user_9995,2020-01-05,telegram / posts,15.75,0.0,2,0.0
9902,user_9995,2020-01-21,yandex / cpc,8.50,0.0,2,0.0


In [22]:
# определяем порядковый номер визита
session_data['sessionNumber'] = session_data.groupby('userId').cumcount() + 1
session_data

,userId,date,trafficSource,cost,value,totalSessions,totalValue,sessionNumber
0,user_10,2020-01-05,telegram / posts,15.75,215.0,1,215.0,1
1,user_10000,2020-01-17,yandex / cpc,8.50,0.0,2,0.0,1
2,user_10000,2020-01-19,google / cpc,8.25,0.0,2,0.0,2
3,user_1002,2020-01-03,telegram / posts,15.75,0.0,1,0.0,1
4,user_1003,2020-01-08,google / cpc,8.25,0.0,2,0.0,1
...,...,...,...,...,...,...,...,...
9899,user_999,2020-01-03,facebook / video,10.25,0.0,1,0.0,1
9900,user_9992,2020-01-04,google / cpc,8.25,0.0,1,0.0,1
9901,user_9995,2020-01-05,telegram / posts,15.75,0.0,2,0.0,1
9902,user_9995,2020-01-21,yandex / cpc,8.50,0.0,2,0.0,2


In [23]:
# сливаем два датасета по колонке пользователи с левой от нее стороны
session_data = session_data.merge(purchases_only, on='userId', how='left')
session_data

,userId,date,trafficSource,cost,value,totalSessions,totalValue,sessionNumber,purchaseDate
0,user_10,2020-01-05,telegram / posts,15.75,215.0,1,215.0,1,2020-01-05
1,user_10000,2020-01-17,yandex / cpc,8.50,0.0,2,0.0,1,NaT
2,user_10000,2020-01-19,google / cpc,8.25,0.0,2,0.0,2,NaT
3,user_1002,2020-01-03,telegram / posts,15.75,0.0,1,0.0,1,NaT
4,user_1003,2020-01-08,google / cpc,8.25,0.0,2,0.0,1,NaT
...,...,...,...,...,...,...,...,...,...
9899,user_999,2020-01-03,facebook / video,10.25,0.0,1,0.0,1,NaT
9900,user_9992,2020-01-04,google / cpc,8.25,0.0,1,0.0,1,NaT
9901,user_9995,2020-01-05,telegram / posts,15.75,0.0,2,0.0,1,NaT
9902,user_9995,2020-01-21,yandex / cpc,8.50,0.0,2,0.0,2,NaT


Теперь посчитаем веса для каждого визита в зависимости от количества дней до момента покупки, чтобы потом использовать эти веса в time-decay модели:

In [25]:
# в сет добавляем колонку с расчетом дней от первого косания до покупки
session_data['daysToPurchase'] = [(x - y).days if x else 0
                                  for x, y in zip(session_data['purchaseDate'], session_data['date'])]
session_data

,userId,date,trafficSource,cost,value,totalSessions,totalValue,sessionNumber,purchaseDate,daysToPurchase
0,user_10,2020-01-05,telegram / posts,15.75,215.0,1,215.0,1,2020-01-05,0.0
1,user_10000,2020-01-17,yandex / cpc,8.50,0.0,2,0.0,1,NaT,NaN
2,user_10000,2020-01-19,google / cpc,8.25,0.0,2,0.0,2,NaT,NaN
3,user_1002,2020-01-03,telegram / posts,15.75,0.0,1,0.0,1,NaT,NaN
4,user_1003,2020-01-08,google / cpc,8.25,0.0,2,0.0,1,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...
9899,user_999,2020-01-03,facebook / video,10.25,0.0,1,0.0,1,NaT,NaN
9900,user_9992,2020-01-04,google / cpc,8.25,0.0,1,0.0,1,NaT,NaN
9901,user_9995,2020-01-05,telegram / posts,15.75,0.0,2,0.0,1,NaT,NaN
9902,user_9995,2020-01-21,yandex / cpc,8.50,0.0,2,0.0,2,NaT,NaN


Time-decay модель. Веса для модели расчитываются по формуле:

вес сессии =  2**−d/s 

где d — дней до покупки s — количество сессий у пользователя

Почему такая формула? Давайте посмотрим на график функции

y =  2**−x/s 

Мы хотим увидеть, как меняется вес сессии в зависимости от 2 факторов:

времени до сессии с покупкой
количества сессий у пользователя

In [26]:
# добавляем колонку с рассечот весы сессии
session_data['timeDecayWeight'] = [2**(-x / y)
                                   for x, y in zip(session_data['daysToPurchase'], session_data['totalSessions'])]
session_data

,userId,date,trafficSource,cost,value,totalSessions,totalValue,sessionNumber,purchaseDate,daysToPurchase,timeDecayWeight
0,user_10,2020-01-05,telegram / posts,15.75,215.0,1,215.0,1,2020-01-05,0.0,1.0
1,user_10000,2020-01-17,yandex / cpc,8.50,0.0,2,0.0,1,NaT,NaN,NaN
2,user_10000,2020-01-19,google / cpc,8.25,0.0,2,0.0,2,NaT,NaN,NaN
3,user_1002,2020-01-03,telegram / posts,15.75,0.0,1,0.0,1,NaT,NaN,NaN
4,user_1003,2020-01-08,google / cpc,8.25,0.0,2,0.0,1,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
9899,user_999,2020-01-03,facebook / video,10.25,0.0,1,0.0,1,NaT,NaN,NaN
9900,user_9992,2020-01-04,google / cpc,8.25,0.0,1,0.0,1,NaT,NaN,NaN
9901,user_9995,2020-01-05,telegram / posts,15.75,0.0,2,0.0,1,NaT,NaN,NaN
9902,user_9995,2020-01-21,yandex / cpc,8.50,0.0,2,0.0,2,NaT,NaN,NaN


In [29]:

session_data['timeDecayWeight'] = session_data['timeDecayWeight'] / session_data.groupby('userId')['timeDecayWeight'].transform(lambda x: x.sum())
session_data.head()

,userId,date,trafficSource,cost,value,totalSessions,totalValue,sessionNumber,purchaseDate,daysToPurchase,timeDecayWeight
0,user_10,2020-01-05,telegram / posts,15.75,215.0,1,215.0,1,2020-01-05,0.0,1.0
1,user_10000,2020-01-17,yandex / cpc,8.50,0.0,2,0.0,1,NaT,NaN,NaN
2,user_10000,2020-01-19,google / cpc,8.25,0.0,2,0.0,2,NaT,NaN,NaN
3,user_1002,2020-01-03,telegram / posts,15.75,0.0,1,0.0,1,NaT,NaN,NaN
4,user_1003,2020-01-08,google / cpc,8.25,0.0,2,0.0,1,NaT,NaN,NaN


И наконец посчитаем ценность каждого визита пользователя согласно разным моделям атрибуции

In [30]:
# расчет выручки на основе различных моделей атрибуции

session_data['lastTouchValue'] = session_data['value']
session_data['firstTouchValue'] = [x if y == 1 else 0
                                   for x, y in zip(session_data['totalValue'], session_data['sessionNumber'])]
session_data['linearValue'] = session_data['totalValue'] / session_data['totalSessions']
session_data['timeDecayValue'] = session_data['totalValue'] * session_data['timeDecayWeight']
session_data

,userId,date,trafficSource,cost,value,totalSessions,totalValue,sessionNumber,purchaseDate,daysToPurchase,timeDecayWeight,lastTouchValue,firstTouchValue,linearValue,timeDecayValue
0,user_10,2020-01-05,telegram / posts,15.75,215.0,1,215.0,1,2020-01-05,0.0,1.0,215.0,215.0,215.0,215.0
1,user_10000,2020-01-17,yandex / cpc,8.50,0.0,2,0.0,1,NaT,NaN,NaN,0.0,0.0,0.0,NaN
2,user_10000,2020-01-19,google / cpc,8.25,0.0,2,0.0,2,NaT,NaN,NaN,0.0,0.0,0.0,NaN
3,user_1002,2020-01-03,telegram / posts,15.75,0.0,1,0.0,1,NaT,NaN,NaN,0.0,0.0,0.0,NaN
4,user_1003,2020-01-08,google / cpc,8.25,0.0,2,0.0,1,NaT,NaN,NaN,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9899,user_999,2020-01-03,facebook / video,10.25,0.0,1,0.0,1,NaT,NaN,NaN,0.0,0.0,0.0,NaN
9900,user_9992,2020-01-04,google / cpc,8.25,0.0,1,0.0,1,NaT,NaN,NaN,0.0,0.0,0.0,NaN
9901,user_9995,2020-01-05,telegram / posts,15.75,0.0,2,0.0,1,NaT,NaN,NaN,0.0,0.0,0.0,NaN
9902,user_9995,2020-01-21,yandex / cpc,8.50,0.0,2,0.0,2,NaT,NaN,NaN,0.0,0.0,0.0,NaN


мы видим, что атрибуция по последнему касанию – это просто наша ценность, которая получена в варианте по умолчанию, это то, как считают в первую очередь. Ценность по первому касанию смотрит на порядковый номер визита и назначает визиту с номером 1 всю выручку от пользователя. Линейная модель просто делит всю выручку пользователя на количество визитов и каждому визиту присваивает это значение. А модель time-decay общую ценность пользователя распределяет на каждый визит пропорционально посчитанному ранее весу. Выше есть объяснение, как работают веса этой модели, если вы еще этого не сделали, то ознакомьтесь с ним

Проверим наши расчеты:

In [31]:
# проверка корректности расчетов

print(session_data[['lastTouchValue', 'firstTouchValue', 'linearValue', 'timeDecayValue']].sum())

lastTouchValue     131345.0
firstTouchValue    131345.0
linearValue        131345.0
timeDecayValue     131345.0
dtype: float64


Суммарная выручка должна быть одинакова по всем моделям, мы ведь не можем взять какую-то выручку извне или куда-то потерять часть средств.

In [32]:
# финальная таблица

totals = session_data.groupby('trafficSource')[['cost', 'lastTouchValue', 'firstTouchValue', 'linearValue',
                                                'timeDecayValue']].sum()
totals

,cost,lastTouchValue,firstTouchValue,linearValue,timeDecayValue
trafficSource,,,,,
facebook / video,28208.00,33735.0,38547.5,34870.541667,34155.463580
google / cpc,25137.75,42557.5,41110.0,42677.333333,44794.862614
telegram / posts,35374.50,29330.0,25460.0,27610.208333,27862.806077
vk / display,4498.75,4345.0,4652.5,4116.833333,3544.756254
yandex / cpc,14025.00,21377.5,21575.0,22070.083333,20987.111474


В результате мы можем посчитать ROI для каждого из способов атрибуции:

In [33]:
totals['lastTouchROI'] = 100*(round(totals['lastTouchValue'] / totals['cost'], 4))
totals['firstTouchROI'] = 100*(round(totals['firstTouchValue'] / totals['cost'], 4))
totals['linearROI'] = 100*(round(totals['linearValue'] / totals['cost'], 4))
totals['timeDecayROI'] = 100*(round(totals['timeDecayValue'] / totals['cost'], 4))

totals[['cost', 'lastTouchROI', 'firstTouchROI', 'linearROI', 'timeDecayROI']]

,cost,lastTouchROI,firstTouchROI,linearROI,timeDecayROI
trafficSource,,,,,
facebook / video,28208.00,119.59,136.65,123.62,121.08
google / cpc,25137.75,169.30,163.54,169.77,178.20
telegram / posts,35374.50,82.91,71.97,78.05,78.77
vk / display,4498.75,96.58,103.42,91.51,78.79
yandex / cpc,14025.00,152.42,153.83,157.36,149.64


**Ответьте на вопросы:**

- как зависит ценность сессии от количества сессий у пользователя в линейной модели атрибуции?
    **чем больше сессий, тем меньше ценность одного визита**
- какая сессия для нас ценнее, согласно модели time-decay, которая произошла сутки назад или которая произошла две недели назад? 
    **ценнее более свежая сессия, которая произошла сутки назад**
- в каком случае ценность на одну сессию снижается сильнее, когда у пользователя 5 сессий или когда у него 10 сессий, для модели time-decay?
    **когда у пользователя 5 сессий, потому что количество сессий находится в знаменателе дроби или потому что во втором случае мы делим ценность на бОльшее количество частей и перепады в ценностях будут ниже**